In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import sqlite3
from datetime import date
from datetime import datetime as dt
import hashlib
import sys
import os
import locale
import re
import requests as rq
import wbdata as wb
from contextlib import contextmanager
import pytz
from datetime import timedelta

project_dir = '/home/mi/Dropbox/prog/python/tradeDB'
os.chdir(project_dir)

from importlib import reload
from workers import sql
from workers import api
from workers.common import read_json
import tradeDB

In [3]:
SECTORS = {
            "INDEXES": {"file": "./assets/indexes.jsonc"},
            "STOCK": {"file": "./assets/stock.jsonc"},
            "ETF": {"file": "./assets/etf.jsonc"},
            "COMODITIES": {"file": "./assets/comodities.jsonc"},
        }
for k in SECTORS:
    SECTORS[k]["data"] = read_json( # type: ignore
        SECTORS[k]["file"]
    )

In [59]:
search=pd.read_csv("./dev/tmp.csv")
search = search.iloc[:,0].to_list()
tab = "STOCK_DESC"
db = './trader.sqlite'
get = ['hash', 'symbol', 'name', 'country', 'industry', 'info', 'from_date', 'to_date']
cols=['symbol']

In [6]:
trade= tradeDB.Trader()

In [8]:
reload(sql)

<module 'workers.sql' from '/home/mi/Dropbox/prog/python/tradeDB/workers/sql.py'>

In [50]:
all_cols = sql.tab_columns(tab=tab, db_file=trade.db)
tab = tab.upper()
search = [s.upper() for s in search]
get = [g.lower() for g in get]
cols = [c.lower() for c in cols]
if cols[0] == "%":
    cols = all_cols
if get[0] == "%":
    get = all_cols
if not all(g in all_cols for g in get):
    print(f"Not correct get='{get}' argument.")
    print(f"possible options: {all_cols}")

# check if tab exists!
if not sql.tab_exists(tab):
    print("tab do not exists")


In [93]:
serach_split = sql.__split_list__(search, 500)

resp_col = pd.DataFrame(columns=get)
resp={}

for c in cols:
    for search_part in serach_split:
        cmd = f"SELECT {','.join(get)} FROM {tab} WHERE "
        cmd += " ".join([f"{c} LIKE '{s}' OR " for s in search_part])
        cmd += f"{c} LIKE 'none'"  # just to close last 'OR'
        resp_part = sql.__execute_sql__([cmd], trade.db)
        resp_col = pd.concat([resp_col,resp_part[cmd]], ignore_index=True)
    resp[c] = resp_col.drop_duplicates()


In [95]:
resp['symbol']

,hash,symbol,name,country,industry,info,from_date,to_date
0,37c1286f9330d090df9ccf2bc7255645,UAL.US,UNITED AIRLINES HOLDINGS,US,None,None,2023-09-29,2023-09-29
1,30b42e636626388f6ec6a6a92775d266,XEL.US,XCEL ENERGY,US,None,None,2023-09-29,2023-09-29
2,0962db69f611cff544c426582f319181,AAPL.US,APPLE,US,None,None,2023-09-29,2023-09-29
3,6c3b002af94ba91b3431701d1b41c974,HON.US,HONEYWELL INT,US,None,None,2023-09-29,2023-09-29
4,d0990407bd978092025ccf5efbc78163,EXPD.US,EXPEDITORS INT OF WASHINGTON,US,None,None,2023-09-29,2023-09-29
5,8bd60948ac47a74a98777491c05340cf,JBHT.US,JB HUNT TRANSPORT SERVICES,US,None,None,2023-09-29,2023-09-29
6,8efbc63814e0bcd88131699e5de95002,CSCO.US,CISCO SYSTEMS,US,None,None,2023-09-29,2023-09-29
7,8531ab7f64bb30255079b4e913b2caab,CSX.US,CSX,US,None,None,2023-09-29,2023-09-29
8,7082601d4aa11910ad3f610af0f4c347,JBLU.US,JETBLUE AIRWAYS,US,None,None,2023-09-29,2023-09-29
9,bc5a27ddd418ceb83d540df3ee493840,LSTR.US,LANDSTAR SYSTEM,US,None,None,2023-09-29,2023-09-29
